## Algoritmo de Verlet

Sea $\vec{r}(t)$ la posición de una partícula en el instante t, entonces su forma en serie de Taylor evaluada en $t+\Delta t$:

\begin{equation}
\label{a}
\vec{r}(t+ \Delta t) = \vec{r}(t) + \vec{r}'(\vec{r},t)\Delta t + \vec{r}''(\vec{r},t)\frac{\Delta t^2}{2} + \cdots
\end{equation}

Y en $t - \Delta t$:
\begin{equation}
\label{b}
\vec{r}(t - \Delta t) = x(t) - \vec{r}'(x,t)\Delta t + \vec{r}''(\vec{r},t)\frac{\Delta t^2}{2} - \cdots
\end{equation}

Sumando ambas ecuaciones:
\begin{equation}
\vec{r}(t+\Delta t) = 2\vec{r}(t) - \vec{r}(t-h) + \Delta t^2 \vec{r}''(\vec{r},t)
\end{equation}
 

Haciendo $h = \Delta t$, $a(\vec{r},t) = \vec{r}''(\vec{r},t)$, $\vec{r}_{i-1}=\vec{r}(t-\Delta t)$, $\vec{r}_i = \vec{r}(t)$, $\vec{r}_{i+1}=\vec{r}(t+\Delta t)$, se obtiene la expresión discretizada del algoritmo de Verlet:

$$\vec{r}_{i+1} = 2\vec{r}_i - \vec{r}_{i-1} + h^2a(\vec{r}_i,t_i)$$

Para iniciar el algoritmo, haciendo $i=0$, se requiere conocer los valores de $\vec{r}_{-1}$ y $\vec{r}_0$. Como $\vec{r}_0 = \vec{r}(t=0)$ corresponde a la condición inicial que en principio se debe conocer, de la ecuación \ref{b} se tiene:

\begin{equation}
\label{verlet_no_disipativo}
\vec{r}_{-1} = \vec{r}_0 - \vec{v}_0h + \frac{h^2}{2}\vec{a}(\vec{r}_0,t_0)
\end{equation}


## Extensión para fuerzas disipativas linealmente dependientes de la velocidad:

Sea una fuerza disipativa con una componente dependiente de la posición y otra de la velocidad de forma lineal:
$$\vec{F} = \vec{F^r} + \vec{F^v} = \vec{F^r} - \alpha\vec{v}$$

Expresando a la velocidad como la derivada de la posición en una aproximación por diferencias finitas centrales:

$$\vec{V}(t) = \frac{\vec{r}(t+h) - \vec{r}(t-h)}{2h}$$

Entonces por la ecuación \ref{verlet_no_disipativo}, se tiene:

\begin{equation}
r_{i+1} = 2r_{i} - r_{i-1} + \frac{h^2}{m}\vec{F^r} + \frac{h^2}{m}\vec{F^v}
\end{equation}

\begin{equation}
\implies \vec{r}_{i+1} = 2\vec{r}_{i} - \vec{r}_{i-1} + \vec{a}_{i}h^2 - \frac{\alpha h}{2m}\left[\vec{r}_{i+1} - \vec{r}_{i-1} \right]
\end{equation}

de donde se obtiene la expresión del algoritmo de Verlet para fuerzas dispativas dependientes linealmente de la velocidad en 3D:
\begin{equation}
\vec{r}_{i+1} = \frac{2m}{\alpha h + 2m }\left[ 2\vec{r}_i + \left(\frac{\alpha h}{2m} - 1 \right)\vec{r}_{i-1} + h^2\vec{a}_{i} \right]
\end{equation}


# Movimiento de Varias Cargas con Interacción Electrostática en un Medio Resistivo

Simular el movimiento de n partículas de carga q que se mueven en un medio resistivo dentro de un círculo de m partículas fijas de carga Q.

La fuerza que actúa sobre cada partícula es de la forma 
$$m\vec{a}_i = \vec{F}_i = K\left[ q^2\sum_{j\neq i}\frac{1}{r^2_j}\hat{r}_j + qQ\sum_n\frac{1}{r^2_n}\hat{r}_n \right]$$

O simplemente:
$$\vec{a}_i = \sum_{j\neq i}K\frac{q_iq_j}{m_j}\frac{1}{r^2_{ij}}\hat{r}_{ij}$$
 
![Title](images/cargas.png)


In [4]:
#Parámetros de gráficas
!jt -t onedork -tfs 14 -fs 16 -cursw 2 -cursc r -cellw 90% -T -mathfs 110 -lineh 150
plot_theme = 'chesterish'

from numpy import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as p
from mpl_toolkits.mplot3d import Axes3D; 
import pandas as pd
import math

#Parámetros de Gráficas
#parámetros matplotlib
from jupyterthemes import jtplot
jtplot.style(theme=plot_theme, context = 'talk', ticks = True, grid = False, fscale = 2)


if plot_theme == 'chesterish' or plot_theme == 'onedork': color_text = 'white'
elif plot_theme == 'grade3': color_text = 'black'
    
plt.rcParams.update({'text.usetex': True,'font.family' : 'serif', 'font.weight' : 'bold',
                     'text.color' : color_text, 'axes.labelcolor' : color_text,
                     'xtick.color' : color_text, 'ytick.color' : color_text,
                     'figure.dpi' : 100, 'savefig.format' : 'jpg', 'savefig.bbox' : 'tight',
                     'lines.linewidth': 5,
#                      'axes.titlesize' : 32, 'font.size' : 10
                    })


#plotly params
import plotly.graph_objects as go
import plotly.io as pio
pio.templates

if plot_theme == 'chesterish' or plot_theme == 'onedork': template = 'plotly_dark'
elif plot_theme == 'grade3': template = 'seaborn'

In [5]:
    from vpython import *
    from random import random

    class particula:
        def __init__(self,indice, r1 = np.zeros(3), carga=100, col=color.red):
            self.indice = indice
            self.r1=r1
            self.carga=carga
            self.color=col
            self.tray = [r1]
            
        r0 = np.zeros(3); r2 = np.zeros(3)
        v = np.zeros(3); 
        masa = 1.0
        
        def a(self):
            a = np.zeros(3)
            K = 1.
            for p in sistema:
                if (p.indice!=self.indice):
                    r_norm = np.linalg.norm(self.r1 - p.r1)
                    r̂ = self.r1 - p.r1
                    a = a + (K/p.masa)*(self.carga*p.carga/ r_norm**3)*r̂
            return a

In [6]:
    α=0.3; f=100; mov=10
    tf = 200; Niter = 500
    
    dt = tf/Niter
    
    radio = 100
    fijas=[particula(i+mov, r1=np.array([radio*cos(2*pi*i/f), radio*sin(2*pi*i/f), 0]), col=color.yellow, carga = 100) for i in range(f)]
    moviles=[particula(i, r1=np.array([80*random()*cos(2*pi*random()), 80*random()*sin(2*pi*random()) , 0]), carga=90)  for i in range(mov)]
    sistema=moviles+fijas
    
    for p in sistema: p.r0 = p.r1 - p.v*dt + 0.5*(dt**2)*p.a() 
    
    for i in range(Niter):
        for p in moviles:
            p.r2 = (2*p.masa/(α*dt+2*p.masa)) *(2*p.r1 + (α*dt/2/p.masa-1)*p.r0 + p.a() * dt**2)
            p.tray.append(p.r2)
            p.r0 = p.r1; p.r1=p.r2       

In [12]:
    scene = canvas(range = 130, width=900, height=900, background=color.black)

    esferas=[sphere(pos=vec(p.r1[0],p.r1[1],0), radius=1.5, color=p.color) for p in sistema]    
    for i in range(Niter):        
        rate(50)
#         if (int(i * 100)) % 5 == 0:
#             scene.capture('im' + str(int(i)))
        for p in moviles:
            esferas[p.indice].pos = vec(p.tray[i][0], p.tray[i][1], p.tray[i][2] )

<IPython.core.display.Javascript object>